In [1]:
import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
from items import Item
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from openai import OpenAI

d:\HealthCare_AI_Assistant_system\wound_analysis\henv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\HealthCare_AI_Assistant_system\wound_analysis\henv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shrian Singh\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3.1-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pyt

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', '')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', '')

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\Shrian Singh\.cache\huggingface\token
Login successful


In [4]:
openai = OpenAI()

In [5]:
# Let's avoid curating all our data again! Load in the pickle files:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [6]:
# OpenAI recommends fine-tuning with populations of 50-100 examples
# But as our examples are very small, I'm suggesting we go with 500 examples (and 1 epoch)

fine_tune_train = train[:100]
fine_tune_validation = train[100:150]

## Step 1
##### Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [7]:
# First let's work on a good prompt for a Frontier model
# Notice that I'm removing the " to the nearest dollar"
# When we train our own models, we'll need to make the problem as easy as possible, 
# but a Frontier model needs no such simplification.

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": f"Price is ${item.price:.2f}"}
    ]

In [8]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [9]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [10]:
write_jsonl(fine_tune_train, "fine_tune_train.jsonl")

In [11]:
write_jsonl(fine_tune_validation, "fine_tune_validation.jsonl")

In [12]:
with open("fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [13]:
train_file

FileObject(id='file-RWTco4ZLDhxNtmxdWYaiWJ3g', bytes=97039, created_at=1729248766, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [14]:
with open("fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [15]:
validation_file

FileObject(id='file-oXTiQBQn55qANtQbz8VM7RUW', bytes=48052, created_at=1729248786, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## Step 2
#### NOW ITS TIME FINE TUNING

In [16]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    seed=42,
    hyperparameters={"n_epochs": 1},
    suffix="pricer"
)

FineTuningJob(id='ftjob-MXIj43gI91euQSqSNvmnFqGL', created_at=1729249747, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-iumMGS1Wm1CV21idNVA5Y6on', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-RWTco4ZLDhxNtmxdWYaiWJ3g', validation_file='file-oXTiQBQn55qANtQbz8VM7RUW', estimated_finish=None, integrations=[], user_provided_suffix='pricer')

In [17]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-MXIj43gI91euQSqSNvmnFqGL', created_at=1729249747, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-iumMGS1Wm1CV21idNVA5Y6on', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-RWTco4ZLDhxNtmxdWYaiWJ3g', validation_file='file-oXTiQBQn55qANtQbz8VM7RUW', estimated_finish=None, integrations=[], user_provided_suffix='pricer')], object='list', has_more=False)

In [18]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [19]:
job_id

'ftjob-MXIj43gI91euQSqSNvmnFqGL'

In [20]:
openai.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-MXIj43gI91euQSqSNvmnFqGL', created_at=1729249747, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-iumMGS1Wm1CV21idNVA5Y6on', result_files=[], seed=42, status='running', trained_tokens=None, training_file='file-RWTco4ZLDhxNtmxdWYaiWJ3g', validation_file='file-oXTiQBQn55qANtQbz8VM7RUW', estimated_finish=None, integrations=[], user_provided_suffix='pricer')

In [22]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-0wcmrNhmn3CjkjlHKSQAUNGq', created_at=1729250090, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-Q9h8N5nwD2005weaRH8lGFnU', created_at=1729250087, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-oMcW9Fj7DS9K5RC5PrDmoGcO', created_at=1729250076, level='info', message='Step 100/100: training loss=1.02, validation loss=0.78, full validation loss=0.71', object='fine_tuning.job.event', data={'step': 100, 'train_loss': 1.016002893447876, 'valid_loss': 0.7818808555603027, 'total_steps': 100, 'full_valid_loss': 0.7096808433532715, 'train_mean_token_accuracy': 0.75, 'valid_mean_token_accuracy': 0.75, 'full_valid_mean_token_accuracy': 0.8375}, type='metrics'),
 FineTuningJobEvent(id='ftevent-Wb1ClPYnB80C1gtA9yjHDbzL', created_at=1729250071, level='info', message='Step 

### Lets use our fine tuned model

In [ ]:
fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model

In [ ]:
# The prompt

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:

def gpt_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name, 
        messages=messages_for(item),
        seed=42,
        max_tokens=7
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
print(test[0].price)
print(gpt_fine_tuned(test[0]))